In [1]:
## QICK LIBRARY
from qick import *

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger

import qick.tprocv2_assembler as qasm
from importlib import reload  # Python 3.4+

import numpy as np
import time

##### Load FGPA BitStream
soc = QickSoc('./qick_111_nur.bit')
for iTile in soc.dac_tiles:
    print("DAC tile %d locked: %s"%(iTile, soc.rf.dac_tiles[iTile].PLLLockStatus == 2))
for iTile in soc.adc_tiles:
    print("ADC tile %d locked: %s"%(iTile, soc.rf.adc_tiles[iTile].PLLLockStatus == 2))
print(soc)
soc.qtt.info()

DAC tile 1 locked: True
ADC tile 0 locked: True

QICK configuration:

	Board: ZCU111

	Software version: 0.2.266
	Firmware timestamp: Fri Jun 21 15:13:06 2024

	Global clocks (MHz): tProcessor 409.600, RF reference 204.800

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (10.000 us)
		fs=6553.600 MHz, fabric=409.600 MHz, 32-bit DDS, range=6553.600 MHz
		DAC tile 1, blk 1 is DAC229_T1_CH1 or RF board output 5
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (10.000 us)
		fs=6553.600 MHz, fabric=409.600 MHz, 32-bit DDS, range=6553.600 MHz
		DAC tile 1, blk 3 is DAC229_T1_CH3 or RF board output 7

	0 readout channels:

	4 digital output pins:
	0:	PMOD0_0
	1:	PMOD0_1
	2:	PMOD0_2
	3:	PMOD0_3

	tProc qick_processor ("v2") rev 20: program memory 256 words, data memory 256 words
		external start pin: None
---------------------------------------------
 QICK Time Tagger INFO 
---------------------------------------------
Connections:
 ADC0 : ADC tile 0,

In [40]:
import xrfdc

# this sets tile 0 block 0 threshold 0 to hysteresis mode, the same interface is used to set/read threshold levels
#print(soc.rf.adc_tiles[0].blocks[0].thresholds[0].Settings)
#print(soc.rf.adc_tiles[0].blocks[0].thresholds[1].Settings)

thresh_settings = soc.rf.adc_tiles[0].blocks[0].thresholds[0].Settings.copy()
print("OLD settings:",thresh_settings)
thresh_settings = soc.rf.adc_tiles[0].blocks[0].thresholds[1].Settings.copy()
print("OLD settings:",thresh_settings)

thresh_settings['ThresholdMode']     = xrfdc.TRSHD_HYSTERISIS
thresh_settings['ThresholdAvgVal']   = 100
thresh_settings['ThresholdUnderVal'] = 100
thresh_settings['ThresholdOverVal']  = 100
soc.rf.adc_tiles[0].blocks[0].thresholds[0].Settings = thresh_settings

thresh_settings['ThresholdMode']     = xrfdc.TRSHD_STICKY_UNDER
thresh_settings['ThresholdAvgVal']   = 200
thresh_settings['ThresholdUnderVal'] = 200
thresh_settings['ThresholdOverVal']  = 200
soc.rf.adc_tiles[0].blocks[0].thresholds[1].Settings = thresh_settings


print("NEW settings:",soc.rf.adc_tiles[0].blocks[0].thresholds[0].Settings)
print("NEW settings:",soc.rf.adc_tiles[0].blocks[0].thresholds[1].Settings)

# this sets tile 0 block 0 threshold 0 to manual clear
soc.rf.adc_tiles[0].blocks[0].thresholds[0].SetClrMode(xrfdc.THRESHOLD_CLRMD_MANUAL_CLR)

# this calls XRFdc_ThresholdStickyClear() with tile 0 block 0 threshold 0
soc.rf.adc_tiles[0].blocks[0].thresholds[0].StickyClear()

OLD settings: {'ThresholdMode': 3, 'ThresholdAvgVal': 5, 'ThresholdUnderVal': 100, 'ThresholdOverVal': 100}
OLD settings: {'ThresholdMode': 3, 'ThresholdAvgVal': 5, 'ThresholdUnderVal': 200, 'ThresholdOverVal': 200}
NEW settings: {'ThresholdMode': 3, 'ThresholdAvgVal': 100, 'ThresholdUnderVal': 100, 'ThresholdOverVal': 100}
NEW settings: {'ThresholdMode': 2, 'ThresholdAvgVal': 200, 'ThresholdUnderVal': 200, 'ThresholdOverVal': 200}


In [154]:
reload(qasm)
asm = """
//TEST program

.ALIAS counter r0

TIME inc_ref #10000

// Set Waveform
REG_WR w_freq   imm #430000
REG_WR w_gain   imm #30000
REG_WR w_conf   imm #17 // PHRST   - NoPeriodic - DDS
REG_WR w_phase  imm #0
REG_WR w_env    imm #0
REG_WR w_length imm #2500


// OUT Waveform
WPORT_WR p0 r_wave
WPORT_WR p1 r_wave

// Time Increment
REG_WR s14 op -op(s14 + #40) 

// OUT Pulses
TRIG p0 set
TRIG p1 clr
TRIG p2 clr

// Time Increment
REG_WR s14 op -op(s14 + #600) 
TRIG p0 clr
TRIG p1 set
TRIG p2 clr

// Time Increment
REG_WR s14 op -op(s14 + #600) 
TRIG p0 clr
TRIG p1 clr
TRIG p2 set

// Time Increment
REG_WR s14 op -op(s14 + #600) 
TRIG p0 clr
TRIG p1 clr
TRIG p2 clr

END: 
    JUMP END

"""

# this sets tile 0 block 0 threshold 0 to manual clear
soc.rf.adc_tiles[0].blocks[0].thresholds[0].SetClrMode(xrfdc.THRESHOLD_CLRMD_AUTO_CLR)
soc.rf.adc_tiles[0].blocks[0].thresholds[1].SetClrMode(xrfdc.THRESHOLD_CLRMD_AUTO_CLR)
#THRESHOLD_CLRMD_MANUAL_CLR = 0x1
#THRESHOLD_CLRMD_AUTO_CLR   = 0x2
#TRSHD_OFF                  = 0x0
#TRSHD_STICKY_OVER          = 0x1
#TRSHD_STICKY_UNDER         = 0x2
#TRSHD_HYSTERISIS           = 0x3


#thresh_settings['ThresholdMode']     = xrfdc.TRSHD_HYSTERISIS
#thresh_settings['ThresholdMode']     = xrfdc.TRSHD_HYSTERISIS
thresh_settings['ThresholdMode']     = xrfdc.TRSHD_HYSTERISIS

thresh_settings['ThresholdAvgVal']   = 10
thresh_settings['ThresholdUnderVal'] = 2500
thresh_settings['ThresholdOverVal']  = 2500
soc.rf.adc_tiles[0].blocks[0].thresholds[0].Settings = thresh_settings

thresh_settings['ThresholdMode']     = xrfdc.TRSHD_HYSTERISIS
thresh_settings['ThresholdAvgVal']   = 10
thresh_settings['ThresholdUnderVal'] = 3500
thresh_settings['ThresholdOverVal']  = 3500
soc.rf.adc_tiles[0].blocks[0].thresholds[1].Settings = thresh_settings


time.sleep(1)

p_txt, p_bin             = Assembler.str_asm2bin(asm)
soc.tproc.Load_PMEM(p_bin)

soc.tproc.start()
time.sleep(1)
#soc.tproc.stop()


In [ ]:
soc.tproc.print_axi_regs()

In [ ]:
soc.tproc.start()
soc.tproc.print_status()
soc.tproc.stop()
soc.tproc.print_debug()

In [ ]:
soc.qtt.print_axi_regs()

In [ ]:
print(soc.qtt.tag0_qty)

In [ ]:
soc.qtt.print_debug()

In [ ]:
soc.qtt.info()

In [ ]:
#tag0 = soc.qtt.read_mem('TAG0',100)

In [ ]:
soc.qtt.get_config()

In [ ]:
soc.qtt.set_threshold(100)
soc.qtt.set_dead_time(10)


In [ ]:
soc.qtt.print_debug()

In [ ]:
soc.qtt.arm()
soc.qtt.disarm()

In [ ]:
soc?

In [ ]:
a = soc.qtt.pop_dt()
print(a)

In [ ]:
print(soc)

In [ ]:
soc.qtt.get_config()

In [ ]:
soc.qtt.set_threshold(32768)
soc.qtt.set_dead_time(100)

In [ ]:
soc.qtt.print_debug()

In [ ]:
soc.qtt.print_axi_regs()

In [ ]:
print(soc.qtt.read_mem('TAG0', 65535))
soc.qtt.print_axi_regs()

In [ ]:
soc.qick_com_0?

In [ ]:
soc.qick_com_0.print_axi_regs()

In [ ]:
soc.qick_com_0.send_byte(15, 2)